## Setup environment

In [7]:

from monai.utils import first, set_determinism
from monai.transforms import (EnsureChannelFirstd, Compose, CropForegroundd, LoadImaged, Orientationd, RandCropByPosNegLabeld, ScaleIntensityRanged, Spacingd)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset
from monai.apps import download_and_extract
from monai.transforms import CenterSpatialCropd
from monai.transforms import Resized
import torch
import matplotlib.pyplot as plt
import os
import glob
import torch.nn as nn
import json
from datetime import datetime
from data_preparation2 import DataHandling 
from UNet_model import create_unet
import numpy as np
import nibabel as nib

In [2]:
data_dir = '/home/shahpouriz/Data/Practic/ASC-PET-001'
directory = '/home/shahpouriz/Data/Practic/LOG'
output_dir = '/home/shahpouriz/Data/Practic/OUT'

In [11]:
import os
import glob
import nibabel as nib
import numpy as np

def calculate_adcm(nasc_img, asc_img, epsilon=1):
    adcm = np.where(nasc_img > epsilon, asc_img / nasc_img, asc_img)
    return adcm

data_dir = '/home/shahpouriz/Data/Practic/ASC-PET-001'
adcm_dir = '/home/shahpouriz/Data/Practic/ASC-PET-001/ADCM'  # Directory to save ADCM images

if not os.path.exists(adcm_dir):
    os.makedirs(adcm_dir)

train_images = sorted(glob.glob(os.path.join(data_dir, "NAC", "*.nii.gz")))
target_images = sorted(glob.glob(os.path.join(data_dir, "MAC", "*.nii.gz")))

for nac in train_images:
    nasc_img = nib.load(nac).get_fdata()

    
    adcm = calculate_adcm(nasc_img)
    adcm_nii = nib.Nifti1Image(adcm, affine=np.eye(4))

    # Saving ADCM image
    adcm_filename = os.path.join(adcm_dir, os.path.basename(nac))
    nib.save(adcm_nii, adcm_filename)


TypeError: calculate_adcm() missing 1 required positional argument: 'asc_img'

In [8]:
# New loading process for using ADCM images
adcm_images = sorted(glob.glob(os.path.join(adcm_dir, "*.nii.gz")))

data_dicts = [{"image": img, "target": tar} for img, tar in zip(adcm_images, target_images)]

train_images = sorted(glob.glob(os.path.join(data_dir, "ADCM", "*.nii.gz")))
target_images = sorted(glob.glob(os.path.join(data_dir, "MAC", "*.nii.gz")))

In [9]:
train_files, val_files =  data_dicts[:20], data_dicts[-5:]

In [10]:
crop_size = (168, 168, 168)  # Adjusted based on my data

train_transforms = Compose(
    [   LoadImaged(keys=["image", "target"]),
        EnsureChannelFirstd(keys=["image", "target"]),
        # ScaleIntensityRanged(keys=["image", "target"],a_min=-60, a_max=120, b_min=0.0, b_max=1.0, clip=True),
        # ReferenceBasedNormalizeIntensityd
        # Orientationd(keys=["image", "target"], axcodes="RAS"),
        Spacingd(keys=["image", "target"], pixdim=(1.5, 1.5, 2.0)),
        Resized(keys=["image", "target"], spatial_size=crop_size, mode='bilinear'),
        CenterSpatialCropd(keys=["image", "target"], roi_size=crop_size),
    ])

val_transforms = Compose(
    [   LoadImaged(keys=["image", "target"]),
        EnsureChannelFirstd(keys=["image", "target"]),
        # ScaleIntensityRanged(keys=["image", "target"],a_min=-60, a_max=120, b_min=0.0, b_max=1.0, clip=True),
        # Orientationd(keys=["image", "target"], axcodes="RAS"),
        Spacingd(keys=["image", "target"], pixdim=(1.5, 1.5, 2.0)),
        Resized(keys=["image", "target"], spatial_size=crop_size, mode=('bilinear')),
        CenterSpatialCropd(keys=["image", "target"], roi_size=crop_size),

    ])

train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=1)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=1)

val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=1)

ValueError: num_samples should be a positive integer value, but got num_samples=0